# Perform an experiment in MLFlow
- start experiment 
- train model
- save metrics
- save artifact

In [ ]:
!pip install mlflow==1.13.1 boto3 awscli -q

In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")
np.random.seed(40)

In [ ]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
data.head()

In [ ]:
data.shape

In [ ]:
TARGET_COLUMN = "quality"
train, test = train_test_split(data)

train_x = train.drop([TARGET_COLUMN], axis=1)
test_x = test.drop([TARGET_COLUMN], axis=1)
train_y = train[[TARGET_COLUMN]]
test_y = test[[TARGET_COLUMN]]

In [ ]:
!aws --endpoint-url $MLFLOW_S3_ENDPOINT_URL s3 mb s3://mlflow

In [ ]:
experiment_id = mlflow.create_experiment(name="My Wine Experiments")

In [ ]:
def experiment(alpha, l1_ratio):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name='wine_models', experiment_id=experiment_id):
            mlflow.set_tag("author", "bpk")
            lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            pred_y = lr.predict(test_x)
            mlflow.log_metric("rmse", np.sqrt(mean_squared_error(test_y, pred_y)))
            mlflow.log_metric("r2", r2_score(test_y, pred_y))
            mlflow.log_metric("mae", mean_absolute_error(test_y, pred_y))

            signature = infer_signature(test_x, pred_y)
            mlflow.sklearn.log_model(lr, "model", registered_model_name="wine-elasticnet", signature=signature)
            
experiment(0.5, 0.5)
experiment(1, 0)
experiment(0, 1)